In [ ]:
import sys
sys.path.append("../source")

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Arrow, OpenHead, NormalHead, VeeHead, Range1d
output_notebook()

from Match_Analytics import Match
from Tracking_Visualization import draw_pitch
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
import pandas as pd
import pickle
import os
working_dir = os.getcwd()

In [ ]:
def Get_Arrows(corner):
    corner_np = corner.to_numpy()
    arrs = np.where(corner_np == 1)
    arrows = []
    for i in range(len(arrs[0])):
        if i == len(arrs[0])-1:
            continue
        else:
            arrows.append([arrs[0][i], arrs[1][i]])
    return np.array(arrows)


def Get_Ball_Position(ball_row):
    poss = np.where(ball_row == 1)
    pos = np.array([poss[0][0], poss[1][0]])
    position = np.array([poss[1][0]])
    return position

def plot_arrows(pitch, colors, means, n_components, color, lab, x_std, y_std, angle, color_ellipse, label=True, ax=None):
    source = ColumnDataSource(dict(x = means[:,0], y = means[:,1]))
    pitch.ellipse(x=means[:,0], 
                    y=means[:,1], 
                    width=x_std, 
                    height=y_std,
                    fill_color = None,
                    line_width = 1.3,
                  line_alpha = 0.8,
                    line_color = color_ellipse,
                    angle = -angle)
    return pitch

def plot_ball(pitch, color, mean, n_components, lab, x_std, y_std, angle, label=True, ax=None):
    pitch.image_url(url=["pilota.png"], x=mean[0][0], y=mean[0][1], w=2.0, h=2.0)

    return pitch
    
    
    

def Plot_TEAM_Corners(pj_frame, b_frame, ini, fin, ball, corner_number, arrow_color, arrow_head):
    
    cov_i = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_ini.npy", "rb"))
    g_i = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_ini.npy", "rb"))
    pos_i = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_ini.npy", "rb"))
    means_i = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_ini.npy", "rb"))

    cov_f = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_fin.npy", "rb"))
    g_f = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_fin.npy", "rb"))
    pos_f = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_fin.npy", "rb"))
    means_f = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_fin.npy", "rb"))
    
    cov_b = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/cov_ball.npy", "rb"))
    g_b = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/gauss_ball.npy", "rb"))
    pos_b = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/pos_ball.npy", "rb"))
    means_b = np.load(open(f"sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/mean_ball.npy", "rb"))
  
    corner = pickle.load(open(working_dir+f"/sampled/pj{pj_frame}_ball{b_frame}_{ini}_{fin}_{ball}/corner_{corner_number}.bin", "rb"))

    ball_pos = Get_Ball_Position(corner[-1:])
    values_i, vectors_i = np.linalg.eig(cov_i)
    x_std_i = np.sqrt(values_i[:,0])
    y_std_i = np.sqrt(values_i[:,1])
    angle_i = np.arctan(vectors_i[:,0,1]/vectors_i[:,0,0])

    values_f, vectors_f = np.linalg.eig(cov_f)
    x_std_f = np.sqrt(values_f[:,0])
    y_std_f = np.sqrt(values_f[:,1])
    angle_f = np.arctan(vectors_f[:,0,1]/vectors_f[:,0,0])
    
    ball_val, ball_vec = np.linalg.eig(cov_b)
    x_std_b = np.sqrt(ball_val[:,0])
    y_std_b = np.sqrt(ball_val[:,1])
    angle_b = np.arctan(ball_vec[:,0,1]/ball_vec[:,0,0])

    arrows = Get_Arrows(corner)
    colors = plt.cm.twilight_shifted(np.linspace(0.1,0.9,ini))
    pitch = draw_pitch(pattern="stripes_horizontal_vertical", stripes_number = 5, grass_alpha=0.6)
    pitch = plot_arrows(pitch, colors, means_i, ini, "red", "Initial", x_std_i, y_std_i, angle_i, "royalblue")
    colors = plt.cm.twilight_shifted(np.linspace(0.1,0.9,fin))
    pitch = plot_arrows(pitch, colors, means_f, fin, "blue", "Final", x_std_f, y_std_f, angle_f, "crimson")
    pitch = plot_ball(pitch, "yellow", means_b[ball_pos], ball, "Ball", x_std_b[ball_pos], y_std_b[ball_pos], angle_b[ball_pos])
    
    for i in range(len(arrows)): 
        pitch.add_layout(Arrow(end=OpenHead(size=arrow_head, line_color=arrow_color, line_width=0.8), x_start=means_i[:,0][arrows[:,0]][i],
                         y_start=means_i[:,1][arrows[:,0]][i], x_end=means_f[:,0][arrows[:,1]][i], y_end=means_f[:,1][arrows[:,1]][i],
                              line_width=0.9))
    
    show(pitch)

In [ ]:
arrow_color = "black"
size_arrow_head = 5
number_of_corners = 45
for i in range(number_of_corners):
    Plot_TEAM_Corners(25, 25, 6, 6, 6, i+1, arrow_color, size_arrow_head)